In [1]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import json
import numpy as np
import datetime

from sklearn.preprocessing import OneHotEncoder

In [2]:
load_dotenv()
opensea_api_key = os.getenv('OPENSEA_API_KEY')

# Data for Context and Analysis

### Collection Stats

In [3]:
url = "https://api.opensea.io/api/v2/collections/pudgypenguins/stats"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

collection_stats_response_json = response.json()

collection_totals_dict = collection_stats_response_json["total"]

# I had to set this index because there is not index and it was causing an error
# I used ChatGPT to debug this issue
collection_totals_df = pd.DataFrame(collection_totals_dict, index=[0])

collection_totals_df

,volume,sales,average_price,num_owners,market_cap,floor_price,floor_price_symbol
0,345948.65696,78961,4.38126,4778,103133.836319,13.0,ETH


## Trait Counts

In [4]:
url = "https://api.opensea.io/api/v2/traits/pudgypenguins"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

# Formatting as json
traits_json = response.json()

# get the "counts" and not "categories" key
traits_json = traits_json.get("counts", [])

# preview
traits_json

{'Background': {'Beige': 1152,
  'Blue': 1587,
  'Mint': 1389,
  'Red': 755,
  'Tangerine': 1135,
  'Yellow': 873,
  'Pink': 710,
  'Purple': 1282,
  'On The Beach': 1,
  'Supermarket': 1,
  'Green': 1,
  'Trick Or Treating': 1,
  'Underwater': 1},
 'Skin': {'Light Gray': 1189,
  'Mint': 423,
  'Maroon': 731,
  'Normal': 2649,
  'Leopard Pink': 89,
  'Dark Gray': 1329,
  'Baby Pink': 623,
  'Cream': 646,
  'Olive Green': 705,
  'Red': 344,
  'Gold': 44,
  'Leopard Gray': 89,
  'Ice': 22,
  'Navy Blue': 4,
  'Black': 1},
 'Body': {'Vote 4 Pudgy': 203,
  'Kimono Blue': 108,
  'Scarf Blue': 237,
  'Shirt Blue': 290,
  'Fish Lover': 70,
  'Turtleneck Grey': 210,
  'Lei Pink': 168,
  'Bow Tie Blue': 154,
  'Tank Top Yellow': 193,
  'Puffer Orange': 158,
  'Huddle Shirt': 228,
  'Kimono Brown': 268,
  'Lei Assorted': 76,
  'Bronze Medal': 193,
  'Gold Medal': 58,
  'Heart': 102,
  'Turtleneck Green': 254,
  'Scarf Pink': 177,
  'Surfboard Necklace': 166,
  'Swordman': 62,
  'Christmas Sweate

#### All Traits

In [5]:
# df of all traits
# documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html
# similar to how OneHotEncoder creates new column names
traits_df_normalized = pd.json_normalize(traits_json)

traits_df_normalized

,Background.Beige,Background.Blue,Background.Mint,Background.Red,Background.Tangerine,Background.Yellow,Background.Pink,Background.Purple,Background.On The Beach,Background.Supermarket,...,Head.Sideways Red,Head.Egg Gold,Head.Flower Crown,Head.Fish Orange,Head.Ice Crown,Head.Hatched Gold,Head.Pineapple Suit,Head.Banana Suit,Head.Normal,Head.Shark Suit
0,1152,1587,1389,755,1135,873,710,1282,1,1,...,90,45,66,44,14,9,1,1,1,1


#### Isolating Traits

In [6]:
# show all trait keys
traits_json.keys()

dict_keys(['Background', 'Skin', 'Body', 'Face', 'Head'])

In [7]:
# explode the df
traits_df_exploded =  pd.DataFrame(traits_json)
traits_df_exploded = traits_df_exploded.explode("Background")

# create new dfs for each trait, sorted by values
# if you don't do reset_index(drop=True), it appears as a series
background_trait_df = traits_df_exploded["Background"].dropna().sort_values(ascending=False).reset_index()
skin_trait_df = traits_df_exploded["Skin"].dropna().sort_values(ascending=False).reset_index()
body_trait_df = traits_df_exploded["Body"].dropna().sort_values(ascending=False).reset_index()
face_trait_df = traits_df_exploded["Face"].dropna().sort_values(ascending=False).reset_index()
head_trait_df = traits_df_exploded["Head"].dropna().sort_values(ascending=False).reset_index()

# rename columns and set indices
background_trait_df = background_trait_df.rename(columns={"index":"background", "Background":"count"}).set_index("background")
skin_trait_df = skin_trait_df.rename(columns={"index":"skin", "Skin":"count"}).set_index("skin")
body_trait_df = body_trait_df.rename(columns={"index":"body", "Body":"count"}).set_index("body")
face_trait_df = face_trait_df.rename(columns={"index":"face", "Face":"count"}).set_index("face")
head_trait_df = head_trait_df.rename(columns={"index":"head", "Head":"count"}).set_index("head")

# display trait dataframes
display(background_trait_df, skin_trait_df, body_trait_df, face_trait_df, head_trait_df)

,count
background,
Blue,1587.0
Mint,1389.0
Purple,1282.0
Beige,1152.0
Tangerine,1135.0
Yellow,873.0
Red,755.0
Pink,710.0
On The Beach,1.0


,count
skin,
Normal,2649.0
Dark Gray,1329.0
Light Gray,1189.0
Maroon,731.0
Olive Green,705.0
Cream,646.0
Baby Pink,623.0
Mint,423.0
Red,344.0


,count
body,
Shirt Blue,290.0
Kimono Brown,268.0
Shirt Red,259.0
Turtleneck Green,254.0
Scarf Green,250.0
...,...
Pineapple Suit,1.0
Banana Suit,1.0
Mirrored,1.0


,count
face,
Winking,961.0
Blushing,932.0
Normal,864.0
Circle Glasses,768.0
Cute,588.0
Monocle,496.0
Squad,445.0
Eyepatch,426.0
Cross Eyed,417.0


,count
head,
Headband,357.0
Flat Cap Blue,355.0
Bucket Hat Green,349.0
Backwards Hat Red,294.0
Hat Blue,286.0
Top Hat,282.0
Hat Red,279.0
Beanie Orange,276.0
Flat Cap Tan,274.0


# Data for Model

## 30 Days of Events

In [8]:
# Set today's date
today = datetime.datetime.now()

# set number of days in past
past_date = today - datetime.timedelta(days=30)

# Convert to unix (starting 1970!)
unix_past_date = int(past_date.timestamp())

# Create blank df to store page data
events_df = pd.DataFrame()

# Initialize next_event variable
next_event = None

while True:
    url = f"https://api.opensea.io/api/v2/events/collection/pudgypenguins?after={unix_past_date}&event_type=sale"
    if next_event:
        # This adds the next event value (and required syntax) to the URL string
        url += f"&next={next_event}"

    headers = {
        "accept": "application/json",
        "x-api-key": opensea_api_key
    }

    response = requests.get(url, headers=headers)

    # Formatting as json
    events_json = response.json()

    # Get the "asset events"
    events_data = events_json.get("asset_events", [])
    
    # Store to df
    page_events_df = pd.json_normalize(events_data)
    
    # Concatenate to master df
    events_df = pd.concat([events_df, page_events_df], ignore_index=True)
    
    # Update next_event for pagination
    next_event = events_json.get("next")

    # If there are no more pages, exit the loop
    if not next_event:
        break

# preview
events_df

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft.image_url,nft.metadata_url,nft.opensea_url,nft.updated_at,nft.is_disabled,nft.is_nsfw,payment.quantity,payment.token_address,payment.decimals,payment.symbol
0,sale,0x64f780c62477a1f04e94dfb6fd49a39679de309c68be...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1714487579,1,0x5b468edb7688e9ae6c1fa5a6d2debbef06e92907,0xf07a2439296e07bc4320af924e655a01fb69d89c,0x0b964b4816377286d20fff153d21a71404853c00addf...,1714487579,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-30T14:33:01.866832,False,False,11799999000000000008,0x0000000000000000000000000000000000000000,18,ETH
1,sale,,ethereum,,1714486295,1,0x116986e9c95c0ecf4a99ce82dd48b9044ae44a19,0xf91576dad2b5b597d2da8c0b309935a8e819e5d3,0xac41f45a236ae0d0a6cef6db9e0ae2f0dbc2cf4f380c...,1714486295,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-30T14:11:36.515386,False,False,11679000000000000000,0x0000000000000000000000000000000000000000,18,ETH
2,sale,,ethereum,,1714469075,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x6d02ce0cd50bff383035e1de5c8b2235fb22e4e8,0x62b760acaac2bfdb7358de76915745f64474fee9baf2...,1714469075,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-30T09:24:38.165679,False,False,11150000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
3,sale,,ethereum,,1714468895,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x6d02ce0cd50bff383035e1de5c8b2235fb22e4e8,0x27b6a01d4acc9484ce6c95bdaa235b1104e5ce44b4f5...,1714468895,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-30T09:21:37.570443,False,False,11170000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
4,sale,,ethereum,,1714468283,1,0x2d5b975f8df3edea0f9f9f6743c34c461524d286,0x6d02ce0cd50bff383035e1de5c8b2235fb22e4e8,0xdfadbba286db88e79d9750f4a102475f72dc4ffa1e08...,1714468283,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-30T09:11:26.434012,False,False,11210000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,sale,0x414d93172755fffeebbe2d73b2f0b16545a43fa9dda8...,ethereum,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,1711947983,1,0xe83529399fb373ad7fcb92d4e738c4651c093aac,0x645998e8aa4817ed6c6fe244482d8842ace72c29,0x5df81ebb12f66866b2130e4932265ca0f3fe61f1478f...,1711947983,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-18T03:49:39.292137,False,False,13368300000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
1462,sale,,ethereum,,1711947863,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xa69833b9fda816f1bfc79517e7932e64708df0dd,0x9f5592a53405dde4b9c9e2e6a810c0baf050c540d37a...,1711947863,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-02T23:04:14.322692,False,False,13450000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
1463,sale,,ethereum,,1711947863,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x7a6b049d83554da1175d65be3f6264a74a05ae17,0xc0e6134b5ae45ba658413939651ec4a3483f5be20ae5...,1711947863,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-04T23:42

### Individual NFT Info

#### query parameters from https://docs.opensea.io/reference/get_nft

In [9]:
# PATH PARAMETERS

# note: nft contract is the same for the whole collection

# identify nft contract (public blockchain address)
# this creates an array
protocol_address = events_df["nft.contract"].unique()

# eliminate the null by getting the max of the array
protocol_address = np.max(protocol_address)

# find unique values for chain (currency)
chain = events_df["chain"].unique().tolist()

# write value without list
chain = chain[0]

# token ID in a list for iteration
token_id_list = events_df["nft.identifier"].drop_duplicates().to_list()

In [10]:
# parsing features function
# credit: Sean!

def get_features (nft):
    features = {}
    features["Background"] = nft["traits"][0]["value"]
    features["Skin"] = nft["traits"][1]["value"]
    features["Face"] = nft["traits"][2]["value"]
    features["Head"] = nft["traits"][3]["value"]
    features["Body"] = nft["traits"][4]["value"]
    features["Rarity"] = nft["rarity"]["rank"]
    
    return features

In [11]:
nfts_dict = {}
token_id = token_id_list[0]

for token_id in token_id_list:
    url = f"https://api.opensea.io/api/v2/chain/{chain}/contract/{protocol_address}/nfts/{token_id}"
    headers = {
        "accept": "application/json",
        "x-api-key": opensea_api_key
    }

    response = requests.get(url, headers=headers)

    # Formatting as json
    nfts_json = response.json()
    
    nfts_dict[token_id] = get_features(nfts_json["nft"])

In [12]:
nfts_dict

nfts_df = pd.DataFrame(nfts_dict).transpose()

nfts_df

,Background,Skin,Face,Head,Body,Rarity
2773,Beige,Maroon,Cute,Beanie Orange,Scarf Green,5598
5673,Blue,Normal,Kimono Brown,Handlebar Bear,Top Hat,7747
1997,Blue,Normal,Cute,Silver Medal,Hat Red,8040
6099,Beige,Normal,Blushing,Bucket Hat Green,Scarf Pink,8697
8459,Yellow,Eyepatch,Beanie Gray,Olive Green,Crop Top,1342
...,...,...,...,...,...,...
4116,Beige,Shirt Blue,Blushing,Flat Cap Black,Red,4811
3164,Beige,Normal,Olive Green,Bow Tie Pink,Hat Red,5551
6277,Light Gray,Normal,Tangerine,Fish Green,Tank Top Blue,4945
8859,Normal,Tangerine,Squad,Turtleneck Pink,Jester's Hat,6222


In [13]:
combined_events_df = events_df[["nft.identifier", "closing_date", "payment.quantity"]]

combined_events_df.set_index("nft.identifier", inplace=True)

combined_events_df

,closing_date,payment.quantity
nft.identifier,,
2773,1714487579,11799999000000000008
5673,1714486295,11679000000000000000
1997,1714469075,11150000000000000000
6099,1714468895,11170000000000000000
8459,1714468283,11210000000000000000
...,...,...
2987,1711947983,13368300000000000000
1380,1711947863,13450000000000000000
708,1711947863,13450000000000000000


In [14]:
combined_df = combined_events_df.merge(nfts_df, how="outer", right_index=True, left_index=True)

combined_df.sample(10)

,closing_date,payment.quantity,Background,Skin,Face,Head,Body,Rarity
nft.identifier,,,,,,,,
7731,1713177755,10010000000000000000,Winking,Mint,Normal,Turtleneck Blue,Sideways Red,7600
7488,1713396743,10120000000000000000,Blue,Normal,Turtleneck Green,Circle Glasses,Sideways Blue,7542
1961,1713616115,10040000000000000000,Blue,Backwards Hat Red,Baby Pink,Scarf Green,Monocle,5746
3852,1713139799,10750000000000000000,Normal,Tangerine,Eyepatch,Overalls,Grizzly Bear Hat,6416
6062,1714367807,11910000000000000000,Yellow,Cream,Clout Goggles,Camo Helmet,Ice Coat,1231
1961,1713703715,10140000000000000000,Blue,Backwards Hat Red,Baby Pink,Scarf Green,Monocle,5746
1583,1712907191,11480000000000000000,Blue,Dark Gray,Bow Tie Black,Monocle,Camo Helmet,6664
2664,1713370355,10080000000000000000,Mint,Normal,Beard,Turtleneck Green,Mohawk Purple,7700
7270,1712137823,11540000000000000000,Winking,Normal,Yellow,Huddle Shirt,Flat Cap Blue,8711


## Encoding Cat Data (Training)

In [15]:
combined_df.dtypes

closing_date         int64
payment.quantity    object
Background          object
Skin                object
Face                object
Head                object
Body                object
Rarity              object
dtype: object

#### Change Wei to Eth so that we can convert data types.

In [16]:
# change payment.quantity (Wei) to float
# we did not use int because of the overfloat error - no difference with float (both numerical)
combined_df[["payment.quantity", "Rarity"]] = combined_df[["payment.quantity", "Rarity"]].astype(float)

# check
combined_df.dtypes

closing_date          int64
payment.quantity    float64
Background           object
Skin                 object
Face                 object
Head                 object
Body                 object
Rarity              float64
dtype: object

In [17]:
# encoding features

# create instance of OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# create a list of columns to encode
cat_columns = ["Background", "Skin", "Face", "Head", "Body"]

# encode columns
encoded = encoder.fit_transform(combined_df[cat_columns])

# create a dataframe with the encoded columns and numerical columns
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_columns))

# concat with numerical columns
encoded_df = pd.concat([combined_df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# drop cat columns
encoded_df.drop(columns=cat_columns, inplace=True)

encoded_df.sample(5)

,closing_date,payment.quantity,Rarity,Background_Baby Pink,Background_Backwards Hat Blue,Background_Beige,Background_Blue,Background_Bow Tie Blue,Background_Bucket Hat Tan,Background_Cute,...,Body_Surfboard Necklace,Body_Swordman,Body_Tank Top Blue,Body_Top Hat,Body_Tribal Necklace,Body_Turtleneck Blue,Body_Turtleneck Green,Body_Turtleneck Pink,Body_Viking Helmet,Body_Villain Mask
383,1711947983,1.336830e+19,8263.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
697,1712668751,1.164000e+19,5756.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1713779651,1.021000e+19,4267.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
975,1712959739,1.140000e+19,6444.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
842,1713835103,1.080000e+19,3407.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
978,1713559931,1.019000e+19,7916.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
781,1713140975,1.062000e+19,3454.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581,1713757763,1.030000e+19,5462.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1125,1713762527,1.019000e+19,8530.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,1713233879,1.023000e+19,8045.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Encoded dataframe for current price.

#### Creating a list of NFTs in the collection
In the end, we could have created a list of 0-8887, but in the event that there were inconsistencies, we checked with the API. It does not appear that there are inconsistencies though.

In [21]:
# Create blank df to store page data
nftslist_df = pd.DataFrame()

# Initialize next_event variable
next_event = None

while True:
    url = f"https://api.opensea.io/api/v2/collection/pudgypenguins/nfts"
    if next_event:
        # This adds the next event value (and required syntax) to the URL string
        url += f"?next={next_event}"

    headers = {
        "accept": "application/json",
        "x-api-key": opensea_api_key
    }

    response = requests.get(url, headers=headers)

    # Formatting as json
    nftslist_json = response.json()

    # Get and exclude "next" key
    nftslist_data = nftslist_json.get("nfts")
    
    # Store to df
    page_nftslist_df = pd.json_normalize(nftslist_data)
    
    # Concatenate to master df, isolating "identifier" (aka token_id)
    nftslist_df = pd.concat([nftslist_df, page_nftslist_df["identifier"]], ignore_index=True)
    
    # Update next_event for pagination
    next_event = nftslist_json.get("next")

    # If there are no more pages, exit the loop
    if not next_event:
        break

# create an array of unique values to eliminate potential duplicates
nfts_array = nftslist_df[0].unique()

# create a list for the token_ids
nfts_list = []

# add array items to list
for nft in nfts_array:
    nfts_list.append(nft)
    

display(nfts_list[0:10], len(nfts_list))

['8885',
 '8884',
 '8883',
 '8882',
 '8881',
 '8880',
 '8879',
 '8878',
 '8875',
 '8877',
 '8874',
 '8873',
 '8887',
 '8886',
 '8876',
 '8793',
 '8792',
 '8791',
 '8790',
 '8789',
 '8788',
 '8787',
 '8786',
 '8785',
 '8784',
 '8783',
 '8782',
 '8849',
 '8828',
 '8812',
 '8848',
 '8781',
 '8827',
 '8811',
 '8847',
 '8780',
 '8826',
 '8872',
 '8810',
 '8846',
 '8859',
 '8825',
 '8871',
 '8809',
 '8845',
 '8779',
 '8858',
 '8824',
 '8818',
 '8808',
 '8870',
 '8778',
 '8857',
 '8844',
 '8798',
 '8823',
 '8838',
 '8817',
 '8807',
 '8869',
 '8843',
 '8856',
 '8777',
 '8797',
 '8806',
 '8837',
 '8816',
 '8822',
 '8867',
 '8868',
 '8776',
 '8842',
 '8862',
 '8855',
 '8796',
 '8805',
 '8836',
 '8866',
 '8831',
 '8821',
 '8815',
 '8833',
 '8795',
 '8852',
 '8841',
 '8861',
 '8775',
 '8804',
 '8830',
 '8865',
 '8802',
 '8854',
 '8835',
 '8820',
 '8814',
 '8800',
 '8839',
 '8794',
 '8819',
 '8851',
 '8863',
 '8853',
 '8864',
 '8840',
 '8860',
 '8850',
 '8832',
 '8801',
 '8829',
 '8834',
 '8803',
 

8888

#### Create a dataframe of best offers.

In [22]:
# create URLs list
best_offer_urls = []

# loop through nfts_list to create a list of URLS
for nft in nfts_list:
    new_url = f"https://api.opensea.io/api/v2/offers/collection/pudgypenguins/nfts/{nft}/best"
    
    # add url to list of urls
    best_offer_urls.append(new_url)

In [23]:
# Create blank df to store page data
best_offer_df = pd.DataFrame()

for new_url in best_offer_urls:
    
    url = new_url
    
    headers = {
        "accept": "application/json",
        "x-api-key": opensea_api_key
    }

    response = requests.get(url, headers=headers)
    
    # Extract JSON data from the response
    response_json = response.json()
        
    # Convert JSON data to DataFrame
    ind_offer_df = pd.json_normalize(response_json.get("offers", []))
        
    # Concatenate the individual offer DataFrame to the master DataFrame
    best_offer_df = pd.concat([best_offer_df, ind_offer_df], ignore_index=True)

KeyboardInterrupt: 

## Create static CSV file for model.

In [ ]:
encoded_df.to_csv("encoded_data.csv")